# ESPCN Realtime Webcam Upscaler - Training Script (Windows)

**Project:** ESPCN Realtime Webcam Upscaler  
**Institution:** Universitas Tarumanagara  
**Thesis:** Development Of An ESPCN Deep Learning Model For Real-Time Webcam Video Super-Resolution With Hardware Interference Optimization  
**Platform:** Windows Local Machine  
**Date:** January 2026

## Environment Specification
- **OS:** Windows 11
- **Python:** 3.12
- **CPU:** Intel Core I7-7700 (4 Core / 8 Thread)
- **RAM:** 64GB
- **GPU:** Nvidia P104-100 (8GB VRAM)
- **Framework:** PyTorch 2.x with CUDA support

## Overview
This notebook implements training of the ESPCN (Efficient Sub-Pixel Convolutional Neural Network) model for 2x super-resolution of webcam video streams. The training pipeline includes:
- Dataset preparation with caching for efficiency
- Combined MSE + SSIM loss function
- Mixed precision training with automatic casting
- Model checkpointing with resume capability
- FP16 quantization and ONNX export
- Performance benchmarking

## 1. Library Imports

Import all required libraries for training, data processing, and model optimization.

In [ ]:
# Standard library imports
import time
import math
import glob
import random
from pathlib import Path

# Third-party imports
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.transforms import ToTensor, ToPILImage, Compose, ColorJitter

# Progress bar and utilities
from tqdm import tqdm

# ONNX and inference
import onnxruntime as ort
from pytorch_msssim import ssim

## 2. Training Configuration & Hyperparameters

Configure model architecture, training schedule, and optimization parameters.

In [ ]:
# ========================= CONFIGURATION =========================
# Model Architecture
UPSCALE_FACTOR = 2  # 2x upscaling (e.g., 360p → 720p)

# Training Hyperparameters
BATCH_SIZE = 64  # Batch size for training (adjust based on GPU memory)
EPOCHS = 150  # Total training epochs
PATCH_SIZE = 256  # Size of training patches extracted from HR images
LEARNING_RATE = 1e-4  # Adam optimizer learning rate
LOSS_ALPHA = 0.84  # Weight for MSE loss in combined loss (SSIM weight = 1 - LOSS_ALPHA)

# Hardware & Device
NUM_WORKER = 0  # Number of DataLoader workers (0 = no multiprocessing, set based on CPU cores)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Directory Structure
BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "datasets"
OUTPUTS_DIR = BASE_DIR / "outputs"
CHECKPOINTS_DIR = BASE_DIR / "checkpoints"
CHECKPOINT_FILE = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}.pth"

print(f"Configuration loaded. Device: {DEVICE} | Batch Size: {BATCH_SIZE} | Loss Alpha: {LOSS_ALPHA}")

# ========================= REPRODUCIBILITY =========================
def set_seed(seed=42):
    """
    Set random seeds for reproducibility across NumPy, PyTorch, and CUDA.
    
    Args:
        seed (int): Random seed value
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    # Ensure deterministic behavior (may impact performance)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def seed_worker(worker_id):
    """
    Set seed for DataLoader worker processes.
    
    Args:
        worker_id (int): Worker process ID
    """
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


set_seed()

## 3. Dataset Preparation & Environment Setup

Prepare datasets from DIV2K, Flickr2K, and optional personal datasets. Uses caching for efficiency.

In [ ]:
def prepare_environment_and_datasets(patch_size):
    """
    Prepare training environment and validate dataset structure.
    
    Creates necessary directories and builds a cached list of valid training/validation images.
    Only images larger than or equal to patch_size are included in the training set.
    
    Args:
        patch_size (int): Minimum image dimension required for training patches
        
    Returns:
        tuple: (training_image_paths, validation_image_paths) - lists of valid image file paths
        
    Raises:
        RuntimeError: If no training or validation images are found
    """
    # Create output directories
    CHECKPOINTS_DIR.mkdir(exist_ok=True, parents=True)
    OUTPUTS_DIR.mkdir(exist_ok=True, parents=True)

    CACHE_FILE_TRAIN = OUTPUTS_DIR / "valid_train_paths.txt"

    # ========== Validation Dataset ==========
    # Validation paths are scanned fresh each run (fast operation)
    valid_div2k_dir = DATA_DIR / "DIV2K_valid_HR"
    validation_image_paths = glob.glob(str(valid_div2k_dir / '*.*'))

    # ========== Training Dataset (with Caching) ==========
    # Check if cache exists to avoid re-scanning large training directories
    valid_train_paths = []
    if CACHE_FILE_TRAIN.exists():
        print(f"Loading cached training image paths from {CACHE_FILE_TRAIN}...")
        try:
            with open(CACHE_FILE_TRAIN, 'r') as f:
                valid_train_paths = [line.strip() for line in f if line.strip()]
            print(f"✓ Loaded {len(valid_train_paths)} paths from cache.")
        except Exception as e:
            print(f"⚠ Warning: Could not read cache file: {e}. Rebuilding...")
            valid_train_paths = []

    # If cache miss or invalid, rebuild the training path list
    if not valid_train_paths:
        print(f"Building new training image list from datasets...")
        train_div2k_dir = DATA_DIR / "DIV2K_train_HR"
        train_flickr2k_dir = DATA_DIR / "Flickr2K_HR"
        personal_dir = DATA_DIR / "Personal_HR"

        # Combine DIV2K and Flickr2K datasets
        train_image_paths = glob.glob(str(train_div2k_dir / '*.*')) + \
                            glob.glob(str(train_flickr2k_dir / '*.*'))

        # Optionally add custom personal images
        if personal_dir.exists():
            print("✓ Personal dataset found. Adding to training set.")
            train_image_paths += glob.glob(str(personal_dir / '*.*'))
        else:
            print("ℹ Personal dataset not found. Proceeding without it.")

        print(f"Found {len(train_image_paths)} candidate training images.")
        print(f"Validating images (minimum dimension: {patch_size}px)...")

        def is_image_large_enough(image_path, min_size):
            """Check if image meets minimum size requirements."""
            try:
                img = cv2.imread(str(image_path))
                if img is None:
                    return False
                h, w = img.shape[:2]
                return h >= min_size and w >= min_size
            except Exception as e:
                print(f"  ⚠ Error reading {Path(image_path).name}: {e}")
                return False

        valid_train_paths = [
            p for p in tqdm(train_image_paths, desc="Filtering images")
            if is_image_large_enough(p, patch_size)
        ]

        print(f"✓ Filtered training set: {len(valid_train_paths)}/{len(train_image_paths)} images retained.")

        # Save cache for future runs
        try:
            with open(CACHE_FILE_TRAIN, 'w') as f:
                for path in valid_train_paths:
                    f.write(f"{path}\n")
            print(f"✓ Cached training paths to {CACHE_FILE_TRAIN}")
        except Exception as e:
            print(f"⚠ Warning: Could not write cache file: {e}")

    # Verify datasets are not empty
    if not valid_train_paths or not validation_image_paths:
        raise RuntimeError(
            "\n✗ DATASET ERROR: No training or validation images found.\n"
            f"  Check that the following directories exist and contain images:\n"
            f"  - Training: {DATA_DIR / 'DIV2K_train_HR'}\n"
            f"  - Validation: {DATA_DIR / 'DIV2K_valid_HR'}"
        )

    return valid_train_paths, validation_image_paths

## 4. Loss Function Definition

Defines combined MSE + SSIM loss function for training. This hybrid loss improves both pixel-level accuracy and perceptual quality.

In [ ]:
class CombinedLoss(nn.Module):
    """
    Combined MSE and SSIM loss function for super-resolution training.
    
    Combines pixel-level Mean Squared Error (MSE) loss with perceptual 
    Structural Similarity Index (SSIM) loss for improved visual quality.
    
    Formula:
        Loss = α * MSE + (1 - α) * (1 - SSIM)
    
    Args:
        alpha (float): Weight for MSE loss component. Default: 0.7
                      Set to 0.84 for better perceptual quality in thesis implementation.
    """
    def __init__(self, alpha=0.7):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.mse_loss = nn.MSELoss()
        try:
            self.ssim_loss_fn = ssim
        except ImportError:
            raise ImportError(
                "pytorch-msssim not found. Install with:\n"
                "  pip install pytorch-msssim"
            )

    def forward(self, output, target):
        """
        Calculate combined loss.
        
        Args:
            output (torch.Tensor): Model output, shape (B, C, H, W)
            target (torch.Tensor): Ground truth, shape (B, C, H, W)
            
        Returns:
            tuple: (total_loss, mse_loss, ssim_score)
        """
        # MSE Loss - Pixel-level accuracy
        loss_mse = self.mse_loss(output, target)

        # SSIM Loss - Perceptual similarity (1 - SSIM, where 1 is worst, 0 is best)
        ssim_score = self.ssim_loss_fn(output, target, data_range=1.0, size_average=True)
        loss_ssim = 1 - ssim_score

        # Combined Loss
        total_loss = self.alpha * loss_mse + (1 - self.alpha) * loss_ssim

        return total_loss, loss_mse, ssim_score


def psnr(mse):
    """
    Calculate Peak Signal-to-Noise Ratio (PSNR) from Mean Squared Error.
    
    Formula:
        PSNR = 10 * log10(1 / MSE) dB
    
    Args:
        mse (float): Mean Squared Error value
        
    Returns:
        float: PSNR in decibels (dB)
    """
    return 10 * math.log10(1 / mse) if mse > 0 else float('inf')

## 5. Dataset Classes

PyTorch Dataset classes for training and validation with automatic patch creation and augmentation.

In [ ]:
class TrainingSuperResolutionDataset(Dataset):
    """
    Training dataset for super-resolution with online augmentation.
    
    Generates random patches from high-resolution images, applies geometric and
    color augmentations, then creates corresponding low-resolution patches via
    bicubic downsampling.
    
    Args:
        image_filenames (list): Paths to high-resolution training images
        crop_size (int): Size of patches to extract (will be adjusted to multiple of upscale_factor)
        upscale_factor (int): Upscaling factor (2 for 2x, 4 for 4x, etc.)
    """
    def __init__(self, image_filenames, crop_size, upscale_factor):
        super(TrainingSuperResolutionDataset, self).__init__()
        self.image_filenames = image_filenames
        # Ensure patch size is divisible by upscale factor
        self.crop_size = crop_size - (crop_size % upscale_factor)
        self.upscale_factor = upscale_factor
        self.to_tensor = ToTensor()
        self.scale_factor = 1 / upscale_factor
        
        # Data augmentation pipeline
        self.transform = Compose([
            ToPILImage(),
            ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            ToTensor()
        ])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        """
        Get training sample at given index.
        
        Returns:
            tuple: (low_res_patch, high_res_patch) - both torch.Tensor of shape (3, H, W)
        """
        try:
            # Load HR image
            hr_image = cv2.imread(self.image_filenames[index])
            if hr_image is None:
                raise IOError(f"Failed to load: {self.image_filenames[index]}")
            hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"✗ Error loading {self.image_filenames[index]}: {e}")
            # Fallback to next image
            return self.__getitem__((index + 1) % len(self))

        h, w = hr_image.shape[:2]

        # Random crop
        i = random.randint(0, h - self.crop_size)
        j = random.randint(0, w - self.crop_size)
        hr_patch = hr_image[i:i+self.crop_size, j:j+self.crop_size, :]

        # Random horizontal flip
        if torch.rand(1) > 0.5:
            hr_patch = cv2.flip(hr_patch, 1)

        # Random 90° rotation
        if torch.rand(1) > 0.5:
            angle = random.choice([
                cv2.ROTATE_90_CLOCKWISE,
                cv2.ROTATE_180,
                cv2.ROTATE_90_COUNTERCLOCKWISE
            ])
            hr_patch = cv2.rotate(hr_patch, angle)

        # Apply color augmentation and convert to tensor
        hr_patch_tensor = self.transform(hr_patch)

        # Create LR from augmented HR via bicubic downsampling
        lr_patch_tensor = F.interpolate(
            hr_patch_tensor.unsqueeze(0),
            scale_factor=self.scale_factor,
            mode='bicubic',
            align_corners=False,
            antialias=True
        ).squeeze(0)

        return lr_patch_tensor, hr_patch_tensor


class ValidationSuperResolutionDataset(Dataset):
    """
    Validation dataset for super-resolution (no augmentation).
    
    Loads full validation images and creates corresponding low-resolution
    versions via bicubic downsampling for evaluation.
    
    Args:
        image_filenames (list): Paths to high-resolution validation images
        upscale_factor (int): Upscaling factor (2 for 2x, 4 for 4x, etc.)
    """
    def __init__(self, image_filenames, upscale_factor):
        super(ValidationSuperResolutionDataset, self).__init__()
        self.image_filenames = image_filenames
        self.upscale_factor = upscale_factor
        self.to_tensor = ToTensor()
        self.scale_factor = 1 / upscale_factor

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        """
        Get validation sample at given index.
        
        Returns:
            tuple: (low_res_image, high_res_image) - both torch.Tensor of shape (3, H, W)
        """
        try:
            # Load HR image
            hr_image = cv2.imread(self.image_filenames[index])
            if hr_image is None:
                raise IOError(f"Failed to load: {self.image_filenames[index]}")
            hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"✗ Error loading {self.image_filenames[index]}: {e}")
            return self.__getitem__((index + 1) % len(self))

        h, w = hr_image.shape[:2]
        # Crop to multiple of upscale_factor to avoid artifacts
        w_new = w - (w % self.upscale_factor)
        h_new = h - (h % self.upscale_factor)
        hr_image = hr_image[:h_new, :w_new, :]

        # Convert to tensor
        hr_tensor = self.to_tensor(hr_image)

        # Downsample to LR via bicubic interpolation
        lr_tensor = F.interpolate(
            hr_tensor.unsqueeze(0),
            scale_factor=self.scale_factor,
            mode='bicubic',
            align_corners=False,
            antialias=True
        ).squeeze(0)

        return lr_tensor, hr_tensor

## 6. Model Architecture

Efficient Sub-Pixel Convolutional Neural Network (ESPCN) model for super-resolution.

In [ ]:
class ESPCN(nn.Module):
    """
    Efficient Sub-Pixel Convolutional Neural Network (ESPCN).
    
    A lightweight network for super-resolution using sub-pixel convolution.
    Learns upsampling in the feature space, then uses pixel shuffle for efficient
    spatial upsampling.
    
    Architecture:
        - Conv2d(3 → 64, 5×5) + ReLU
        - Conv2d(64 → 64, 3×3) + ReLU
        - Conv2d(64 → 3*r², 3×3) where r = upscale_factor
        - PixelShuffle(r) → output (3, H*r, W*r)
    
    Args:
        upscale_factor (int): Upscaling factor (2 for 2x, 4 for 4x, etc.)
    """
    def __init__(self, upscale_factor):
        super(ESPCN, self).__init__()
        self.upscale_factor = upscale_factor
        
        # Feature extraction layer (5×5 receptive field)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        
        # Non-linear mapping layer
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        # Sub-pixel convolution layer
        # Output channels = 3 (RGB) * (upscale_factor²) for reshuffling
        self.conv3 = nn.Conv2d(64, 3 * (upscale_factor ** 2), kernel_size=3, padding=1)
        
        # Pixel shuffle for efficient spatial upsampling
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        
        # Activation function
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """
        Forward pass.
        
        Args:
            x (torch.Tensor): Input tensor of shape (B, 3, H, W)
            
        Returns:
            torch.Tensor: Output tensor of shape (B, 3, H*r, W*r) where r = upscale_factor
        """
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pixel_shuffle(self.conv3(x))
        return x

## 7. Main Training Loop

Execute the complete training pipeline with validation, checkpointing, and metric logging.

In [ ]:
if __name__ == "__main__":
    print("\n" + "="*70)
    print("ESPCN TRAINING INITIALIZATION")
    print("="*70)
    
    print(f"\n📊 Device Configuration:")
    print(f"  Device: {DEVICE}")
    print(f"  CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  GPU Count: {torch.cuda.device_count()}")
        print(f"  GPU Name: {torch.cuda.get_device_name(0)}")

    # ========== Prepare Datasets ==========
    print(f"\n📁 Preparing datasets...")
    train_paths, val_paths = prepare_environment_and_datasets(PATCH_SIZE)

    if not train_paths or not val_paths:
        raise RuntimeError("✗ Dataset preparation failed. Check your data directories.")

    # ========== Create PyTorch Datasets ==========
    train_dataset = TrainingSuperResolutionDataset(
        train_paths, crop_size=PATCH_SIZE, upscale_factor=UPSCALE_FACTOR
    )
    val_dataset = ValidationSuperResolutionDataset(
        val_paths, upscale_factor=UPSCALE_FACTOR
    )

    # ========== Create DataLoaders ==========
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKER,
        pin_memory=(DEVICE == "cuda"),
        worker_init_fn=seed_worker if NUM_WORKER > 0 else None
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=NUM_WORKER,
        pin_memory=(DEVICE == "cuda")
    )

    print(f"\n✓ Datasets loaded successfully:")
    print(f"  Training samples: {len(train_dataset)}")
    print(f"  Validation samples: {len(val_dataset)}")
    print(f"  Training batches: {len(train_loader)}")

    # ========== Initialize Model ==========
    model = ESPCN(upscale_factor=UPSCALE_FACTOR)
    if DEVICE == 'cuda' and torch.cuda.device_count() > 1:
        print(f"\n🔗 Using DataParallel with {torch.cuda.device_count()} GPUs")
        model = torch.nn.DataParallel(model)
    model = model.to(DEVICE)

    # Count parameters
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\n🧠 Model: ESPCN x{UPSCALE_FACTOR} ({num_params:,} parameters)")

    # ========== Initialize Training Components ==========
    criterion = CombinedLoss(alpha=LOSS_ALPHA).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    scaler = GradScaler(enabled=(DEVICE == "cuda"))

    # ========== Checkpoint & Logging Setup ==========
    start_epoch = 0
    best_psnr = 0.0
    log_file_path = OUTPUTS_DIR / 'training_log.csv'
    training_log = []

    if CHECKPOINT_FILE.exists():
        print(f"\n↻ Loading checkpoint: {CHECKPOINT_FILE}")
        checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)
        
        # Handle state dict compatibility (torch.compile, DataParallel prefixes)
        original_state_dict = checkpoint['model_state_dict']
        new_state_dict = {}

        # Strip '_orig_mod.' prefix from torch.compile
        needs_prefix_stripping = any(key.startswith('_orig_mod.') for key in original_state_dict.keys())
        if needs_prefix_stripping:
            print("  - Detected '_orig_mod.' prefix (torch.compile). Stripping...")
            for key, value in original_state_dict.items():
                new_key = key[len('_orig_mod.'):] if key.startswith('_orig_mod.') else key
                new_state_dict[new_key] = value
        else:
            new_state_dict = original_state_dict

        # Strip 'module.' prefix from DataParallel
        needs_module_stripping = any(key.startswith('module.') for key in new_state_dict.keys())
        if needs_module_stripping:
            print("  - Detected 'module.' prefix (DataParallel). Stripping...")
            stripped_dict = {}
            for key, value in new_state_dict.items():
                new_key = key[len('module.'):] if key.startswith('module.') else key
                stripped_dict[new_key] = value
            new_state_dict = stripped_dict

        model.load_state_dict(new_state_dict)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_psnr = checkpoint.get('best_psnr', 0.0)
        
        print(f"  ✓ Resumed from epoch {start_epoch} (best PSNR: {best_psnr:.4f} dB)")

        # Load previous training log
        if log_file_path.exists():
            try:
                log_df = pd.read_csv(log_file_path)
                log_df = log_df[log_df['epoch'] < start_epoch]
                training_log = log_df.to_dict('records')
                print(f"  ✓ Loaded {len(training_log)} previous log entries")
            except Exception as e:
                print(f"  ⚠ Could not load log: {e}. Starting fresh.")
                training_log = []
    else:
        print(f"\n▶ Starting training from scratch (no checkpoint found)")
        if log_file_path.exists():
            log_file_path.unlink()
            print(f"  - Cleared old log file")

    # ========== BEGIN TRAINING ==========
    print(f"\n{'='*70}")
    print(f"TRAINING ESPCN x{UPSCALE_FACTOR} | Epochs: {start_epoch+1} → {EPOCHS}")
    print(f"Loss: Combined MSE (α={LOSS_ALPHA}) + SSIM (α={1-LOSS_ALPHA})")
    print(f"{'='*70}\n")

    start_time = time.time()
    for epoch in range(start_epoch, EPOCHS):
        model.train()
        epoch_train_loss = 0.0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch")
        for batch_idx, (lr_images, hr_images) in enumerate(progress_bar):
            lr_images = lr_images.to(DEVICE, non_blocking=(DEVICE == "cuda"))
            hr_images = hr_images.to(DEVICE, non_blocking=(DEVICE == "cuda"))

            optimizer.zero_grad()

            # Forward pass with mixed precision for GPU
            if DEVICE == "cuda":
                with autocast(device_type=DEVICE):
                    outputs = model(lr_images)
                    total_loss, mse, ssim_score = criterion(outputs.float(), hr_images.float())
                
                # Backward pass with gradient scaling
                scaler.scale(total_loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                # CPU: standard forward/backward without mixed precision
                outputs = model(lr_images)
                total_loss, mse, ssim_score = criterion(outputs.float(), hr_images.float())
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

            epoch_train_loss += total_loss.item()
            progress_bar.set_postfix({
                'Loss': f"{total_loss.item():.4f}",
                'MSE': f"{mse.item():.6f}",
                'SSIM': f"{ssim_score.item():.4f}"
            })

        # ========== VALIDATION ==========
        model.eval()
        val_psnr, val_ssim = 0.0, 0.0
        with torch.no_grad():
            for i, (lr_images, hr_images) in enumerate(val_loader):
                lr_images = lr_images.to(DEVICE)
                hr_images = hr_images.to(DEVICE)
                
                output = model(lr_images)
                mse_val = nn.functional.mse_loss(output, hr_images)
                val_psnr += psnr(mse_val.item())
                val_ssim += ssim(output, hr_images, data_range=1.0, size_average=True).item()

                # Save sample output every 5 epochs
                if i == 0 and (epoch + 1) % 5 == 0:
                    sr_pil = ToPILImage()(output.squeeze(0).cpu())
                    output_path = OUTPUTS_DIR / f'val_epoch_{epoch+1:03d}.png'
                    sr_pil.save(output_path)

        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)

        # Log metrics
        training_log.append({
            'epoch': epoch + 1,
            'val_psnr': avg_val_psnr,
            'val_ssim': avg_val_ssim,
            'learning_rate': scheduler.get_last_lr()[0]
        })

        log_df = pd.DataFrame(training_log)
        log_df.to_csv(log_file_path, index=False)

        # Epoch summary
        print(f"Epoch {epoch+1:3d} │ "
              f"PSNR: {avg_val_psnr:7.4f} dB │ "
              f"SSIM: {avg_val_ssim:.6f} │ "
              f"LR: {scheduler.get_last_lr()[0]:.2e}")

        scheduler.step()

        # ========== CHECKPOINT BEST MODEL ==========
        if avg_val_psnr > best_psnr:
            best_psnr = avg_val_psnr
            print(f"  ↑ New best PSNR! ({best_psnr:.4f} dB) Saving checkpoint...")
            
            # Get clean state dict (handle compiled models)
            model_state = model.state_dict()
            needs_stripping = any(key.startswith('_orig_mod.') for key in model_state.keys())
            
            if needs_stripping:
                final_model_state = {
                    key[len('_orig_mod.'):] if key.startswith('_orig_mod.') else key: value
                    for key, value in model_state.items()
                }
            else:
                final_model_state = model_state

            torch.save({
                'epoch': epoch,
                'model_state_dict': final_model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_psnr': best_psnr,
                'config': {
                    'upscale_factor': UPSCALE_FACTOR,
                    'loss_alpha': LOSS_ALPHA,
                    'learning_rate': LEARNING_RATE,
                }
            }, CHECKPOINT_FILE)

    # ========== TRAINING COMPLETE ==========
    total_time = time.time() - start_time
    print(f"\n{'='*70}")
    print(f"✓ TRAINING COMPLETE")
    print(f"{'='*70}")
    print(f"Total time: {total_time/3600:.2f} hours ({total_time/60:.1f} minutes)")
    print(f"Best model: {CHECKPOINT_FILE}")
    print(f"Best PSNR: {best_psnr:.4f} dB")
    print(f"Training log: {log_file_path}\n")

    # ========== PLOT RESULTS ==========
    print("Generating training graphs...")
    plt.figure(figsize=(14, 5))
    
    # PSNR plot
    plt.subplot(1, 2, 1)
    plt.plot(log_df['epoch'], log_df['val_psnr'], marker='o', linewidth=2, markersize=3, color='#2E86AB')
    plt.title('Validation PSNR vs. Epoch', fontsize=12, fontweight='bold')
    plt.xlabel('Epoch', fontsize=11)
    plt.ylabel('PSNR (dB)', fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # SSIM plot
    plt.subplot(1, 2, 2)
    plt.plot(log_df['epoch'], log_df['val_ssim'], marker='o', linewidth=2, markersize=3, color='#A23B72')
    plt.title('Validation SSIM vs. Epoch', fontsize=12, fontweight='bold')
    plt.xlabel('Epoch', fontsize=11)
    plt.ylabel('SSIM', fontsize=11)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(OUTPUTS_DIR / 'training_curves.png', dpi=150, bbox_inches='tight')
    print(f"✓ Curves saved to {OUTPUTS_DIR / 'training_curves.png'}")
    plt.show()

## 8. FP16 Quantization

Convert trained model to half-precision floating point for reduced memory and faster inference.

In [ ]:
print("\n" + "="*70)
print("FP16 QUANTIZATION")
print("="*70)

quant_model = ESPCN(upscale_factor=UPSCALE_FACTOR).to(DEVICE)
checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)

# Load and clean state dict
original_state_dict = checkpoint['model_state_dict']
new_state_dict = {}
needs_prefix_stripping = any(key.startswith('_orig_mod.') for key in original_state_dict.keys())

if needs_prefix_stripping:
    print("  - Stripping '_orig_mod.' prefix...")
    for key, value in original_state_dict.items():
        new_key = key[len('_orig_mod.'):] if key.startswith('_orig_mod.') else key
        new_state_dict[new_key] = value
else:
    new_state_dict = original_state_dict

quant_model.load_state_dict(new_state_dict)
quant_model.eval()
quant_model.half()  # Convert to FP16

quantized_model_path = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}_fp16.pth"
torch.save(quant_model.state_dict(), quantized_model_path)

print(f"✓ Model converted to FP16 and saved: {quantized_model_path}\n")

## 9. ONNX Export

Export trained model to ONNX format for cross-platform inference (Windows, Linux, Mobile, etc.).

In [ ]:
print("\n" + "="*70)
print("ONNX EXPORT")
print("="*70)

EXPORT_FP16 = True
DUMMY_INPUT_SIZE = (1, 3, 720, 1280)  # (batch, channels, height, width)

if EXPORT_FP16:
    print(f"\nExporting FP16 model...")
    onnx_model = ESPCN(upscale_factor=UPSCALE_FACTOR)
    checkpoint_path_fp16 = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}_fp16.pth"
    onnx_model.load_state_dict(torch.load(checkpoint_path_fp16))
    onnx_model.eval().half().to(DEVICE)
    dummy_input = torch.randn(*DUMMY_INPUT_SIZE, device=DEVICE).half()
    onnx_model_path = OUTPUTS_DIR / f"espcn_x{UPSCALE_FACTOR}_fp16.onnx"
else:
    print(f"\nExporting FP32 model...")
    onnx_model = ESPCN(upscale_factor=UPSCALE_FACTOR).to(DEVICE)
    checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)

    # Clean state dict
    original_state_dict = checkpoint['model_state_dict']
    new_state_dict = {}
    needs_prefix_stripping = any(key.startswith('_orig_mod.') for key in original_state_dict.keys())

    if needs_prefix_stripping:
        print("  - Stripping '_orig_mod.' prefix...")
        for key, value in original_state_dict.items():
            new_key = key[len('_orig_mod.'):] if key.startswith('_orig_mod.') else key
            new_state_dict[new_key] = value
    else:
        new_state_dict = original_state_dict

    onnx_model.load_state_dict(new_state_dict)
    onnx_model.eval()
    dummy_input = torch.randn(*DUMMY_INPUT_SIZE, device=DEVICE)
    onnx_model_path = OUTPUTS_DIR / f"espcn_x{UPSCALE_FACTOR}.onnx"

# Export to ONNX
torch.onnx.export(
    onnx_model,
    dummy_input,
    str(onnx_model_path),
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamo=False,
    dynamic_axes={
        'input': {2: 'height', 3: 'width'},
        'output': {2: 'height_out', 3: 'width_out'}
    },
    opset_version=17
)

print(f"✓ Model exported to ONNX: {onnx_model_path}\n")

## 10. Performance Benchmarking

Benchmark PyTorch vs ONNX Runtime inference performance.

In [ ]:
print("="*70)
print("INFERENCE BENCHMARKING")
print("="*70 + "\n")

# Prepare test input
if EXPORT_FP16:
    test_image_np = np.random.randn(1, 3, 720, 1280).astype(np.float16)
else:
    test_image_np = np.random.randn(1, 3, 720, 1280).astype(np.float32)

test_image_torch = torch.from_numpy(test_image_np).to(DEVICE)
if EXPORT_FP16:
    test_image_torch = test_image_torch.half()

# ========== PyTorch Benchmark ==========
print("Benchmarking PyTorch inference (100 iterations)...")
if DEVICE == "cuda":
    torch.cuda.synchronize()

with torch.no_grad():
    # Warmup
    _ = onnx_model(test_image_torch)
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    
    # Timed run
    start_time = time.time()
    for _ in range(100):
        _ = onnx_model(test_image_torch)
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    pytorch_time = (time.time() - start_time) / 100

print(f"✓ PyTorch inference: {pytorch_time * 1000:.4f} ms/image\n")

# ========== ONNX Runtime Benchmark ==========
print("Benchmarking ONNX Runtime inference (100 iterations)...")
session = ort.InferenceSession(
    str(onnx_model_path),
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
)
print(f"  Providers: {session.get_providers()}")

ort_inputs = {session.get_inputs()[0].name: test_image_np}

# Warmup
_ = session.run(None, ort_inputs)

# Timed run
start_time = time.time()
for _ in range(100):
    _ = session.run(None, ort_inputs)
onnx_time = (time.time() - start_time) / 100

print(f"✓ ONNX Runtime inference: {onnx_time * 1000:.4f} ms/image\n")

# ========== Results ==========
print("="*70)
if pytorch_time > 0:
    if onnx_time < pytorch_time:
        speedup = (pytorch_time - onnx_time) / pytorch_time * 100
        print(f"✓ ONNX is ~{speedup:.1f}% faster than PyTorch")
    else:
        slowdown = (onnx_time - pytorch_time) / pytorch_time * 100
        print(f"⚠ ONNX is ~{slowdown:.1f}% slower than PyTorch")
else:
    print("⚠ Could not compute speedup (PyTorch time = 0)")

print("="*70 + "\n")